In [1]:
import pandas as pd

In [2]:
%config Completer.use_jedi = False

In [3]:
df = pd.read_csv(r"D:\data\openparliament\bills_partyvote.csv")

In [4]:
df.head()

,vote,party_id,votequestion_id,id,disagreement
0,N,1,1043,3345,0.000000
1,F,2,1043,3346,0.400000
2,Y,3,1043,3347,0.000000
3,F,4,1043,3348,0.287879
4,F,1,1042,3349,0.360294


create pivot table with index=votequestion_id, columns=party_id, values=disagreement

In [5]:
disagreement_labels = df.pivot_table(index='votequestion_id', columns='party_id', values='disagreement')

In [6]:
disagreement_labels.loc[1043]

party_id
1     0.000000
2     0.400000
3     0.000000
4     0.287879
9          NaN
39         NaN
46         NaN
Name: 1043, dtype: float64

In [13]:
disagreement_labels.mean(axis=0)

party_id
1     0.003521
2     0.002292
3     0.000896
4     0.006075
9     0.003412
39    0.000000
46    0.001261
dtype: float64

Conservatives are most likely to vote against their party at .0035 disagreement. That is still an incredibly low number, and speaks to how whipped the parties are into voting with their party's stances. 

In [14]:
#note: need pure counts

In [15]:
df.disagreement.sort_values(ascending=False)

8864     0.5
16630    0.5
7493     0.5
9272     0.5
7328     0.5
        ... 
11191    0.0
11190    0.0
11189    0.0
11188    0.0
0        0.0
Name: disagreement, Length: 16771, dtype: float64

In [16]:
df[df.party_id == 4].sort_values(by='disagreement', ascending=False)

,vote,party_id,votequestion_id,id,disagreement
8232,F,4,2194,11610,0.483871
3085,F,4,399,4080,0.469697
591,F,4,806,5560,0.468085
9964,F,4,2507,13342,0.467456
10189,F,4,2552,13567,0.464706
...,...,...,...,...,...
5257,Y,4,1565,8607,0.000000
5262,Y,4,1566,8612,0.000000
5267,Y,4,1567,8617,0.000000
5277,Y,4,1569,8627,0.000000


In [17]:
df[df.disagreement>0.1].shape

(155, 5)

In [18]:
df.shape

(16771, 5)

In [19]:
155/16771

0.009242144177449169

Less than 1% of the party votes since Jan 1st 2000 have had any disagreement between party members in a vote relative to the party's position. This is staggering to me, and the dispiriting implications of this are that nobody's mind can be changed from that of the party whip. 

In [20]:
df.head()

,vote,party_id,votequestion_id,id,disagreement
0,N,1,1043,3345,0.000000
1,F,2,1043,3346,0.400000
2,Y,3,1043,3347,0.000000
3,F,4,1043,3348,0.287879
4,F,1,1042,3349,0.360294


In [21]:
disagreed_votes = df[df.disagreement>=0.05]

In [22]:
disagreed_votes.pivot_table(columns='party_id', values='disagreement',aggfunc='count')

party_id,1,2,3,4,9,46
disagreement,62,36,10,84,18,2


In [25]:
df.vote.value_counts()

Y    7962
N    6185
A    2494
F     113
P      17
Name: vote, dtype: int64

what do the codes mean? 
'Y' = Yea
'N' = Nay
'A' = 'affirmative'
'F' = seems to mean negative, nay
'P' seems to mean an affirmative vote, possibly "passed"

In [39]:
df.vote.replace({'Y':1, 'N':0, 'A': 1, 'F':0, 'P':1}, inplace=True)

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

making a pivot table with a float for how the party voted per votequestion - with 1-disagreement for yeas, 0+disagreement for nays

In [33]:
df['outcome'] = 0

In [36]:
def agg_outcome(row):
    if row['vote'] == 1:
        return 1-row['disagreement']

In [37]:
df['outcome'] = df[df.vote==1].vote - df[df.vote==1].disagreement

In [38]:
df.outcome.describe()

count    10473.000000
mean         0.999504
std          0.005902
min          0.854701
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: outcome, dtype: float64

In [43]:
df.vote.fillna(2, inplace=True)

count    16771.000000
mean         0.624471
std          0.484274
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: vote, dtype: float64